# PADRÃO ENTEC

In [16]:
import pandas as pd
import chardet
import matplotlib.pyplot as plt
import os

pasta_atual = os.getcwd()

arquivos = os.listdir(pasta_atual)

arquivos_csv = [arquivo for arquivo in arquivos if arquivo.endswith('.csv')]

quantidade_csv = len(arquivos_csv)

for i in range(quantidade_csv):
    tituloCSV = arquivos_csv[i]

    if len(tituloCSV) == 13:
        nMAR = int(tituloCSV[0])
        nPCS = int(tituloCSV[2])
        nInv = int(tituloCSV[4:6])
        ST = int(tituloCSV[7:9])
    elif len(tituloCSV) == 14:
        nMAR = int(tituloCSV[0:2])
        nPCS = int(tituloCSV[3])
        nInv = int(tituloCSV[5:7])
        ST = int(tituloCSV[8:10])
    
    if nMAR < 10:
        if nInv < 10:
            if ST < 10:
                tituloArquivo = f'MAR0{nMAR}-PCS{nPCS} INV0{nInv} ST0{ST}'
            else:
                tituloArquivo = f'MAR0{nMAR}-PCS{nPCS} INV0{nInv} ST{ST}'
        else:
            if ST < 10:
                tituloArquivo = f'MAR0{nMAR}-PCS{nPCS} INV{nInv} ST0{ST}'
            else:
                tituloArquivo = f'MAR0{nMAR}-PCS{nPCS} INV{nInv} ST{ST}'
    else:
        if nInv < 10:
            if ST < 10:
                tituloArquivo = f'MAR{nMAR}-PCS{nPCS} INV0{nInv} ST0{ST}'
            else:
                tituloArquivo = f'MAR{nMAR}-PCS{nPCS} INV0{nInv} ST{ST}'
        else:
            if ST < 10:
                tituloArquivo = f'MAR{nMAR}-PCS{nPCS} INV{nInv} ST0{ST}'
            else:
                tituloArquivo = f'MAR{nMAR}-PCS{nPCS} INV{nInv} ST{ST}'


    tituloComp = tituloArquivo[0:16]
    irradiancia = 'Irradiância(W/m²)'
    temperatura_celula = 'Temperatura(°C)'

    with open(tituloCSV, 'rb') as f:
        rowdata = f.read()
        result = chardet.detect(rowdata)
        encoding = result['encoding']

    df = pd.read_csv(tituloCSV, encoding=encoding, delimiter=';')

    # Tratando o DataFrame que vai virar tabela
    df_tabela = df.iloc[2:3, 6:12]
    df_tabela.columns = ['Pm(W)', 'Voc(V)', 'Isc(A)', 'Vm(V)', 'Im(A)', 'FF']

    df_irradiancia = df.iloc[6:7, 6].rename(irradiancia)
    # print(df_irradiancia)
    df_temperatura = df.iloc[8:9, 6].rename(temperatura_celula)
    # print(df_temperatura)


    # Concatenar as colunas na ordem desejada
    valor_irradiancia = df_irradiancia.iloc[0]
    # print(valor_irradiancia)
    valor_temperatura = df_temperatura.iloc[0]
    valores_finais = pd.DataFrame({irradiancia: [], temperatura_celula: []})
    df_tabela = pd.concat([df_tabela, valores_finais])
    df_tabela[irradiancia] = None
    df_tabela[temperatura_celula] = None
    df_tabela.loc[2, irradiancia] = valor_irradiancia
    df_tabela.loc[2, temperatura_celula] = valor_temperatura
    # print(df_tabela)

    # Converter o DataFrame em uma string
    tabela = df_tabela.to_string(index=False)
    tabela = tabela.strip()
    tabela = tabela.split('\n')
    tabela_formatada = []

    for linha in tabela:
        linha_sem_espaco = linha.strip()

        colunas = linha_sem_espaco.split(' ')

        linha_formatada = []

        for coluna in colunas:
            coluna_sem_espaco = coluna.strip()
            if coluna_sem_espaco.startswith('(') and coluna_sem_espaco.endswith(')'):
                coluna_sem_espaco = coluna_sem_espaco[1:-1]
            
            linha_formatada.append(coluna_sem_espaco)
        tabela_formatada.append(linha_formatada)

    tabela_sem_vazio = []

    for linha in tabela_formatada:
        linha_sem_vazio = []

        for elemento in linha:
            elemento_sem_espaco = elemento.strip()
            if elemento_sem_espaco != '':
                linha_sem_vazio.append(elemento_sem_espaco)
        
        tabela_sem_vazio.append(linha_sem_vazio)

    dados = tabela_sem_vazio[1]
    if i == 0:
        irrad = dados[6]
        temp = dados[7]

    print(tabela_sem_vazio)

    Pm = float(dados[0])
    Isc = float(dados[2])

    # Tratando o DataFrame que vai gerar o gráfico
    colunas_selecionadas = df.columns[:3]
    df_grafico = df[colunas_selecionadas]
    df_grafico.columns = ['Tensão (V)', 'Corrente (A)', 'Potência (W)']
    print(df_grafico)

    # Plotar a curva IxV com eixo vertical para Corrente (A) e Potência (W)
    fig, ax_grafico = plt.subplots()

    # Definir o tamanho do gráfico
    fig.set_figheight(6)
    fig.set_figwidth(18)

        # Plota Corrente (A) primeiro
    ax_grafico.set_xlabel('Tensão (V)', fontweight='bold')
    plt.xticks(fontweight='bold')
    ax_grafico.set_ylabel('Corrente (A)', fontweight='bold')
    plt.yticks(fontweight='bold')
    ax_grafico.plot(df_grafico['Tensão (V)'], df_grafico['Corrente (A)'], 'r-o', linewidth=2, label='Curva I x V')

        # Adiciona o eixo secundário para Potência (W)
    ax2 = ax_grafico.twinx()
    ax2.set_ylabel('Potência (W)', fontweight='bold')
    plt.yticks(fontweight='bold')
    ax2.plot(df_grafico['Tensão (V)'], df_grafico['Potência (W)'], 'b-o', linewidth=2, label='Curva P x V')

    # Ajustar limites dos eixos x e y
    ax_grafico.set_xlim(0, 1500)
    ax_grafico.set_ylim(0, Isc + 3 if Isc +3 >= 20 else 20)
    ax2.set_ylim(0, Pm + 6000 if Pm + 6000 >= 25000 else 25000)

    # Definir título e legendas
    plt.title(f'Curva IxV - {tituloArquivo}', fontweight='bold')

    # Ajustar a posição da legenda Curva I x V
    ax_grafico.legend(loc='upper right', bbox_to_anchor=(1, 0.9))
    for texto_legenda in ax_grafico.get_legend().get_texts():
        texto_legenda.set_fontweight('bold')
    # Ajustar a posição da legenda Curva P x V
    ax2.legend(loc='upper right', bbox_to_anchor=(1, 0.84))
    for texto_legenda in ax2.get_legend().get_texts():
        texto_legenda.set_fontweight('bold')

    # Plotar a tabela no segundo espaço do gráfico
    table = plt.table(cellText=tabela_sem_vazio, loc='bottom', cellLoc='center', bbox=[0, -0.3, 1, 0.2])
    table.auto_set_font_size(False)
    table.set_fontsize(12)
    table.scale(1.5, 1.5)

    for j in range(2):
        for i in range(len(tabela_sem_vazio[j])):
            cell = table[j, i]
            cell.set_text_props(fontweight='bold')

    plt.subplots_adjust(bottom=0)
    # Salva o arquivo em .jpg
    plt.savefig(f'./imagem/{tituloArquivo}.jpg', bbox_inches='tight', dpi=600)
    plt.show()

[['Pm(W)', 'Voc(V)', 'Isc(A)', 'Vm(V)', 'Im(A)', 'FF', 'Irradiância(W/m²)', 'Temperatura(°C)'], ['23521.89', '1426.04', '22.210', '1134.53', '20.733', '0.743', '1148.43', '45.90']]
     Tensão (V)  Corrente (A)  Potência (W)
0       0.00000      22.21010       0.00000
1      78.17757      22.13000    1730.07007
2      89.36152      22.07744    1972.87329
3     100.63018      22.05500    2219.39893
4     111.90501      22.03913    2466.28931
..          ...           ...           ...
195  1417.33118       1.17618    1667.04053
196  1417.56213       1.14716    1626.16504
197  1417.77173       1.11916    1586.71362
198  1417.97998       1.09199    1548.41882
199  1426.03955       0.00000       0.00000

[200 rows x 3 columns]
[['Pm(W)', 'Voc(V)', 'Isc(A)', 'Vm(V)', 'Im(A)', 'FF', 'Irradiância(W/m²)', 'Temperatura(°C)'], ['22260.16', '1426.55', '20.951', '1137.44', '19.570', '0.745', '1088.60', '49.00']]
     Tensão (V)  Corrente (A)  Potência (W)
0       0.00000      20.95116       0.0000

# Recortar parte específica da imagem

In [17]:
import cv2

img = cv2.imread('ST01.jpeg')

copia = img[780:820, 0:270]
# cv2.imshow('copia', copia)

cv2.imwrite('copia.jpg', copia)
cv2.waitKey(0)
cv2.destroyAllWindows()

# Retirar a data e hora da Imagem

In [18]:
import pytesseract
from PIL import Image
import re

imagem = Image.open('copia.jpg')
imagem = imagem.resize((imagem.width*2, imagem.height*2))
imagem = imagem.convert('L')
imagem = imagem.point(lambda x: 0 if x < 128 else 255, '1')

caminho = r"C:\Users\Darieldon\AppData\Local\Programs\Tesseract-OCR"
pytesseract.pytesseract.tesseract_cmd = caminho + r'\tesseract.exe'

config = '--psm 6'

texto = pytesseract.image_to_string(imagem, config=config)
print(texto)

dataList = re.split(r"\s+",texto)
resultList = [i.strip() for i in dataList if i != '']

print(resultList)

21.06.2024 11:23

['21.06.2024', '11:23']


# Adicionar os dados obtidos para a planilha de protocolo

In [19]:
from openpyxl import Workbook, load_workbook
import xlwings as xw

planilha = xw.Book(f'./Protocolo/Relatório de Ensaio de Curva IxV - {tituloComp}.xlsm')
aba_ativa = planilha.sheets['BD']
aba_ativa.activate()

novaLista = [[irrad], [temp]]

for i, row in enumerate(novaLista):
    print(i, row)
    aba_ativa.range(f'A{i+2}').value = row

planilha.save(f'./Protocolo/Relatório de Ensaio de Curva IxV - {tituloComp}.xlsm')

11:23
0 ['1148.43']
1 ['45.90']
2 ['21.06.2024']
3 ['11:23']
